In [10]:
import os
import mmcv
import torch
import albumentations as A

from mmcv import Config
from mmseg.datasets import build_dataloader, build_dataset
from mmseg.models import build_segmentor
from mmseg.apis import single_gpu_test
from mmcv.runner import load_checkpoint
from mmcv.parallel import MMDataParallel
from pycocotools.coco import COCO
import pickle
import pandas as pd
import numpy as np
import json

In [2]:
# epoch 지정해주기
epoch = 'best_mIoU_epoch_36'
# work dir 설정해주기
work_dir = '/opt/ml/mmsegmentation/work_dirs/upernet_convnext_xlarge_fp16_640x640_160k_ade20k_pseudo_offline/'

# config file 들고오기
# config file은 수정해서 사용
cfg = Config.fromfile('/opt/ml/mmsegmentation/configs/convnext/upernet_convnext_xlarge_fp16_640x640_160k_ade20k_pseudo_offline.py')
root='/opt/ml/input/mmseg/images/test'

# dataset config 수정
cfg.data.test.img_dir = root
# cfg.data.test.pipeline[1]['img_scale'] = (512,512)
cfg.data.test.test_mode = True

cfg.data.samples_per_gpu = 1
cfg.work_dir = work_dir

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.model.train_cfg = None

In [3]:
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=4,
        dist=False,
        shuffle=False)

2022-05-12 09:10:51,762 - mmseg - INFO - Loaded 624 images


In [4]:
# checkpoint path
checkpoint_path = os.path.join(cfg.work_dir, f'{epoch}.pth')

model = build_segmentor(cfg.model, test_cfg=cfg.get('test_cfg'))
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu')
model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

/opt/ml/mmsegmentation/mmseg/models/losses/cross_entropy_loss.py:232: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss over non-ignore labels, which is the same with PyTorch official cross_entropy, set ``avg_non_ignore=True``.
  warnings.warn(


load checkpoint from local path: /opt/ml/mmsegmentation/work_dirs/upernet_convnext_xlarge_fp16_640x640_160k_ade20k_pseudo_offline/best_mIoU_epoch_36.pth


In [5]:
output = single_gpu_test(model, data_loader)

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 624/624, 0.3 task/s, elapsed: 2344s, ETA:     0s

In [24]:
import pickle

out_logit_pkl_path = 'result.pkl'

with open(out_logit_pkl_path, 'wb') as pkl_file:
    logits_to_pickle = np.stack(output, axis=0)  # ndarray (819, 11, 512, 512)
    logits_to_pickle = np.clip(logits_to_pickle * 255., 0., 255.)  # 혹시 몰라서 clamping
    logits_to_pickle = logits_to_pickle.astype(np.uint8)
    pickle.dump(logits_to_pickle, pkl_file, protocol=4)


In [15]:
def load_pkl(pkl_file_path):
    with open(pkl_file_path, 'rb') as pkl_file:
        logit_uint8 = pickle.load(pkl_file)
        
    logit_float32 = logit_uint8.astype(dtype=np.float32) / 255.
    
    return logit_float32

In [19]:
pkl_file_paths = ['/opt/ml/input/code/pkl/result.pkl','/opt/ml/input/code/pkl/result_ksk.pkl','/opt/ml/input/code/pkl/jyj.pickle','/opt/ml/input/code/pkl/knet_convnext.pkl','/opt/ml/input/code/pkl/ngp_result.pkl']

out_csv_file_path = 'final.csv'

out_pkl_file_path = '/opt/ml/input/code/pkl/fin.pkl'

In [20]:
logit_list = list()
for pkl_file_path in pkl_file_paths:
    logit = load_pkl(pkl_file_path)
    print(logit.shape)
    logit_list.append(logit)
    
avg_logit = np.average(np.array(logit_list), axis=0)


with open(out_pkl_file_path, 'wb') as pkl_file:
    logit_to_pickle = np.clip(avg_logit * 255., 0., 255.)  # 혹시 몰라서 clamping
    logit_to_pickle = logit_to_pickle.astype(np.uint8)
    pickle.dump(logit_to_pickle, pkl_file, protocol=4)



(624, 11, 512, 512)
(624, 11, 512, 512)
(624, 11, 512, 512)
(624, 11, 512, 512)
(624, 11, 512, 512)


In [21]:
soft_pred = avg_logit.argmax(axis=1)

In [23]:
resize_transform = A.Compose([A.Resize(256, 256, 0)])  # cv2.INTER_NEAREST

prediction_strings = list()
file_names = list()
coco = COCO("/opt/ml/input/data/test.json")

for i, out in enumerate(list(soft_pred)):
    image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
    file_names.append(image_info['file_name'])
    out_fit_array = resize_transform(image=out)['image']
    
    prediction_string = ' '.join([str(pixel_pred) for pixel_pred in out_fit_array.flatten().tolist()])
    prediction_strings.append(prediction_string)

submission = pd.DataFrame()
submission['image_id'] = file_names
submission['PredictionString'] = prediction_strings
submission.to_csv(out_csv_file_path, index=False)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [24]:
submission.head

<bound method NDFrame.head of                  image_id                                   PredictionString
0    batch_01_vt/0021.jpg  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
1    batch_01_vt/0028.jpg  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
2    batch_01_vt/0031.jpg  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
3    batch_01_vt/0032.jpg  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
4    batch_01_vt/0076.jpg  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
..                    ...                                                ...
619  batch_02_vt/1949.jpg  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
620  batch_02_vt/1960.jpg  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
621  batch_02_vt/1974.jpg  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
622  batch_02_vt/1990.jpg  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
623  batch_02_vt/2006.jpg  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...

[624 rows x 2 columns]>

In [23]:
from tqdm import tqdm
# submission 양식에 맞게 output 후처리
input_size = 512
output_size = 256

submission = pd.read_csv("/opt/ml/input/code/submission/sample_submission.csv", index_col=None)
json_dir = os.path.join("/opt/ml/input/data/test.json")

with open(json_dir, "r", encoding="utf8") as outfile:
    datas = json.load(outfile)

# PredictionString 대입
for image_id, predict in tqdm(enumerate(output)):

    image_id = datas["images"][image_id]
    file_name = image_id["file_name"]

    temp_mask = []
    predict = predict.reshape(1, 11, 512, 512)
    mask = predict.reshape((1, 11, output_size, input_size//output_size, output_size, input_size//output_size)).max(5).max(3) # resize to 256*256
    temp_mask.append(mask)
    oms = np.array(temp_mask)
    oms = oms.reshape([oms.shape[0], 11, output_size*output_size]).astype(np.float16)
    string = oms.flatten()
    submission = pd.concat([submission, pd.DataFrame([{"image_id" : file_name, "PredictionString" : ' '.join(str(e) for e in string.tolist())}])]
                                   , ignore_index=True)


submission.to_csv(os.path.join(cfg.work_dir, f'submission_{epoch}.csv'), index=False)

624it [08:41,  1.20it/s]


KeyboardInterrupt: 